In [1]:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

import os
from pathlib import Path
project_path = Path(os.environ.get(project_environment_variable))

scripts_path = project_path.joinpath("scripts")

import sys
sys.path.append(str(scripts_path)) 

from utils import path_setup
path = path_setup.subfolders(project_path)

path['project'] : D:\projects\skin-lesion-classification
path['images'] : D:\projects\skin-lesion-classification\images
path['models'] : D:\projects\skin-lesion-classification\models
path['expository'] : D:\projects\skin-lesion-classification\expository
path['literature'] : D:\projects\skin-lesion-classification\literature
path['notebooks'] : D:\projects\skin-lesion-classification\notebooks
path['presentation'] : D:\projects\skin-lesion-classification\presentation
path['scripts'] : D:\projects\skin-lesion-classification\scripts


In [29]:
S = {"b", "a", "c","g", "e"}
S.update({"f", "d" })
# S = sorted(S)
for i, x in enumerate(S):
    print(i, x)

0 d
1 c
2 f
3 g
4 a
5 e
6 b


In [2]:
from multiclass_models import process

In [6]:
from typing import Type

# process class attributes
data_dir: Path = path["images"] # Path to directory containing metadata csv file.
filename: str = "metadata.csv"  # Name of metadata csv file.
tvr: int = 3                    # Train/val ratio (number of lesions represented in train set is approx. tvr times number represented in val set).
seed: int = 0                   # Random seed for selection of single image per lesion to be assigned to training set (if keep_first is False).
keep_first: bool = False        # If True, keep first image for each lesion; otherwise, select random image for each lesion.
dxs: list = ["nv",
             "mel",
             "bcc",
             "akiec",]          # Subset of all diagnoses we are interested in classifying (missing dxs will be placed in "other" class).

In [7]:
metadata = process(data_dir, 
                   filename,
                   tvr,
                   seed, 
                   keep_first,
                   dxs,)

Successfully loaded file 'D:\projects\skin-lesion-classification\images\metadata.csv'.
Inserted 'num_images' column.
Inserted 'label' column.
Added 'set' column with tags 't1', 'v1', 'ta', and 'va'.


See ```01_binary_classification.ipynb``` for an explanation of the tags ```t1``` etc.

In [8]:
metadata.df.head(4)

,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set
0,HAM_0000118,2,ISIC_0027419,bkl,other,histo,80.0,male,scalp,va
1,HAM_0000118,2,ISIC_0025030,bkl,other,histo,80.0,male,scalp,v1
2,HAM_0002730,2,ISIC_0026769,bkl,other,histo,80.0,male,scalp,ta
3,HAM_0002730,2,ISIC_0025661,bkl,other,histo,80.0,male,scalp,t1


In [9]:
for across in ["lesions", "images"]:
    for subset in ["all", "train", "val"]:
        process.dx_dist(metadata, subset = subset, across = across)

DISTRIBUTION OF LESIONS BY DIAGNOSIS: OVERALL


dx,nv,other,mel,bcc,akiec
freq,5403.00,898.00,614.00,327.00,228.00
%,72.33,12.02,8.22,4.38,3.05


Total lesions: 7470.

DISTRIBUTION OF LESIONS BY DIAGNOSIS: TRAIN


dx,nv,other,mel,bcc,akiec
freq,4024.00,681.00,476.0,251.00,170.00
%,71.83,12.16,8.5,4.48,3.03


Total lesions: 5602 (74.99% of all lesions).

DISTRIBUTION OF LESIONS BY DIAGNOSIS: VAL


...

Total images: 10015.

DISTRIBUTION OF IMAGES BY DIAGNOSIS: TRAIN


dx,nv,other,mel,bcc,akiec
freq,4999.00,1025.00,865.0,390.00,240.00
%,66.48,13.63,11.5,5.19,3.19


Total images: 7519 (75.08% of all images).

DISTRIBUTION OF IMAGES BY DIAGNOSIS: VAL


dx,nv,other,mel,bcc,akiec
freq,1706.00,331.00,248.00,124.00,87.00
%,68.35,13.26,9.94,4.97,3.49


Total images: 2496 (24.92% of all images).



In [7]:
test_df = metadata.df.sample(n=64,random_state=0)

In [8]:
import torchvision.transforms as transforms

transform = transforms.Compose([
transforms.CenterCrop((300, 300)),
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])    

In [9]:
import pandas as pd
from typing import Type
from typing import List, Callable

df: pd.DataFrame = test_df # metadata.df 
train_set: list = ["t1"]              # ["t1"] (one image per lesion in training set); ["t1", "ta"] (all images for each lesion in training set).
dxs: list = dxs                       # List of diagnoses (labels/targets) to classify.
label_codes: dict = metadata.label_codes
data_dir: Path = path["images"]       # Path to directory where images are stored.
model_dir: Path = path["models"]      # Path to directory where models/model info/model results are stored.
transform: List[Callable] = transform # Transform to be applied to images before feeding to ResNet-18
batch_size: int = 32                  # Mini-batch size: default 32.
epochs: int = 1                       # Number of epochs (all layers unfrozen from the start): default 10.
base_learning_rate: float = 0.001     # Learning rate to start with: default 0.001. Using Adam optimizer.
filename_stem: str = "rn18mc"         # For saving model and related files. train set and num epochs will be appended automatically. Default "rn18mc".
filename_suffix: str = "test"         # Something descriptive and unique for future reference and to avoid over-writing other files. Default empty string "".

In [10]:
from multiclass_models import resnet18

resnet18mc_test = resnet18(df,
                           train_set, 
                           dxs,
                           label_codes,
                           data_dir,
                           model_dir,
                           transform,
                           batch_size,
                           epochs,
                           base_learning_rate,
                           filename_stem,
                           filename_suffix,)

In [11]:
resnet18mc_test.train()

Epoch 1/1, Training Loss: 2.0678, Validation Loss: 1.1020
Saving model.state_dict() as D:\projects\skin-lesion-classification\models\rn18mc_t1_1e_test.pth.
model.state_dict() can now be accessed through state_dict attribute.
Train/val losses can now be accessed through epoch_losses attribute.


In [12]:
resnet18mc_test.epoch_losses

{'train_loss': array([2.06780052]), 'val_loss': array([1.101951])}

In [13]:
prob_df = resnet18mc_test.inference()
prob_df

,image_id,prob_other,prob_nv,prob_mel,prob_bcc,prob_akiec
0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
0,ISIC_0028664,0.062252,0.778029,0.009443,0.097707,0.052569
1,ISIC_0025998,0.079042,0.689123,0.024967,0.074022,0.132846
2,ISIC_0032817,0.159856,0.452721,0.144249,0.063163,0.18001
3,ISIC_0026577,0.147288,0.459583,0.121426,0.116006,0.155697
...,...,...,...,...,...,...
27,ISIC_0027291,0.099387,0.108867,0.04234,0.606721,0.142684
28,ISIC_0026654,0.036743,0.579736,0.093502,0.079474,0.210546
29,ISIC_0033943,0.052816,0.824196,0.02526,0.067377,0.030351
30,ISIC_0033183,0.251628,0.291551,0.092659,0.158193,0.205969


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image

In [ ]:
model = resnet18mc_test.model
model.eval();

In [ ]:
image = Image.open(path["images"].joinpath("ISIC_0028664.jpg"))
image = transform(image)
image = image.unsqueeze(0)  # Add batch dimension (1 image)

In [25]:
# Make a prediction
with torch.no_grad():
    output = model(image)

In [26]:
# Get the predicted class index
_, predicted = torch.max(output, 1)
predicted_class_index = predicted.item()

In [27]:
# Apply softmax to get probabilities
softmax = nn.Softmax(dim=1)
probabilities = softmax(output)
prob = { label : probabilities[0][idx].item() for idx, label in enumerate(metadata.label_codes.keys()) }
# prob_dict = { label : probabilities[0][idx].item() for idx, label in enumerate(self.label_codes.keys()) }
prob

{'other': 0.12805189192295074,
 'nv': 0.6369990110397339,
 'mel': 0.025738338008522987,
 'bcc': 0.03843827173113823,
 'akiec': 0.1707724630832672}

In [32]:
probabilities.numpy().flatten()

array([0.12805189, 0.636999  , 0.02573834, 0.03843827, 0.17077246],
      dtype=float32)

In [21]:
import numpy as np
display(pd.Series(probabilities.numpy().flatten(), name="prob"))
display(pd.Series([probabilities.numpy().flatten()], name="prob"))

0    2.666995e-09
1    1.000000e+00
2    3.777259e-11
3    1.944458e-12
4    4.334258e-08
Name: prob, dtype: float32

0    [2.6669953e-09, 1.0, 3.7772594e-11, 1.9444583e...
Name: prob, dtype: object

In [23]:
image_ids = ["ISIC_0028664"]
batch_image_ids = pd.Series(image_ids, name = "image_id")
# Pair each image_id with its corresponding prediction (probability) and put them in our dataframe
batch_probs = pd.Series([probabilities.cpu().numpy().flatten()], name = "prob")
batch = pd.concat([batch_image_ids, batch_probs], axis = 1,  keys=["image_id", "prob"])
batch

,image_id,prob
0,ISIC_0028664,"[2.6669953e-09, 1.0, 3.7772594e-11, 1.9444583e..."


In [21]:
codes_label = { value : key for (key, value) in metadata.label_codes.items()}
codes_label[predicted_class_index]

'nv'

In [20]:
image_id_prob = pd.DataFrame({"image_id": [pd.NA], "prob": [pd.NA]})
image_id_prob = image_id_prob.dropna()
image_id_prob

,image_id,prob
